In [69]:
%matplotlib inline
import numpy as np
import csv

In [94]:
import csv

#importing the csv file
with open('deficits.csv') as f:
    defmatrix=[list(line) for line in csv.reader(f)]
#defmatrix is a matrix (list of lists) of all rows of dataset

In [95]:
defmatrix2 = []
defmatrix2.append(defmatrix[0])
for row in defmatrix[1:]:
    rowtemp=[]
    rowtemp.append(row[0])
    for x in row[3:]:
        x = float(x)
        rowtemp.append(x)
    defmatrix2.append(rowtemp)

In [7]:
#print(defmatrix2)

In [96]:
services = []
for row in defmatrix2[1:]:
    services.append(row[0])
uniques = set(services)
uniques2 = list(uniques)
print(uniques2)

['420', '405', '400', '600', '506', '309', '251', '411', '111', '200', '122', '253 & 254', '304', '409', '402', '108', '109', '107', '601', '121', '408', '605', '602', '113', '306', '102 & 105', '901 & 902', '505', '510', '100', '250', '303', '404', '106', '406', '407', '509', '604', '900', '301', '702', '202', '307', '703', '252', '500', '800', '305', '700', '603', '424', '118 & 124', '504', '507', '302', '403', '201', '401', '502', '701', '501', '112', '300', '503', '414']


In [98]:
uniqdef = {}
uniqcounter=[]
for item in uniques:
    sums=[0.0]*12
    counter=0
    for row in defmatrix2[1:]:
        if row[0] == item:
            counter+=1
            deficits=[]
            temp = 0
            for value in range(12):
                if temp < 24:
                    curr = getdeficit(row[1:], temp, temp+1)
                    deficits.append(curr)
                    temp+=2
            for value in range(12):
                sums[value]+=deficits[value]
    uniqdef[item] = sums
    uniqcounter.append((item,counter))
print(uniqdef["304"])

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -1259.0, -1284.0, -1307.0]


In [85]:
def getdeficit(row, fund_index, req_index):
    """Returns the deficit of a pair of funding/requirements.
    row: a list of fund/reqs (2010-2021, len(row) = 26)
    fund_index: the index of the desired year's funding in the list
    req_index: the index of the desired year's requirements in the list"""
    
    fund = row[fund_index]
    #print("This is the funding: " + str(fund))
    req = row[req_index]
    #print("This is the requirement: " + str(req))
    
    #UNDER-FUNDING
    if fund < 0.0:
        #print("UNDER")
        deficit = fund
        
    elif ((fund > req) and (req == 0.0) and (fund > 0.0)):
        #print("MET")
        deficit = 0.0
        
    elif ((fund < req) and (fund == 0.0) and (req > 0.0)):
        #print("UNDER")
        deficit = fund-req
        
    elif ((fund > req) and (fund > 0.0) and (req > 0.0)):
        #print("OVER")
        deficit = fund-req
        
    elif ((fund < req) and (fund > 0.0) and (req > 0.0)):
        #print("UNDER")
        deficit = fund-req
        
    elif (fund == req):
        #print("MET")
        deficit = 0.0
        
    return deficit

In [ ]:
test = [-80, 10, -80, 0, 80, 0, 0, 80, 100, 80, 80, 100, 80, 80, 0, 0]

# TESTING

In [75]:
def get_deficit_ratios(iss, row):
    """Returns the ratio for a row of funding/requirements.
    row: a list of fund/reqs (2010-2021, len(row) = 24, pairs = 12)"""
    rowreqs = []
    for item in range(len(row)):
        if (item % 2 != 0):
            rowreqs.append(row[item])
    #print("REQS: " + str(rowreqs))
    
    rowfunds = []
    for item in range(len(row)):
        if (item %2 == 0):
            rowfunds.append(row[item])
    #print("FUNDS: " + str(rowfunds))
    
    rowdefs = []
    ratios = []
    for i in range(12):
        fund = rowfunds[i]
        req = rowreqs[i]

        if fund < 0.0:
            #print("UNDER")
            deficit = fund
            ratio = -((np.absolute(fund)/(np.absolute(fund)+req)))

        elif ((fund > req) and (req == 0.0) and (fund > 0.0)):
            #print("MET")
            deficit = 0.0
            ratio = 0.0

        elif ((fund < req) and (fund == 0.0) and (req > 0.0)):
            #print("UNDER")
            deficit = fund-req
            ratio = deficit/req

        elif ((fund > req) and (fund > 0.0) and (req > 0.0)):
            #print("OVER")
            deficit = fund-req
            ratio = (deficit)/req
            #FLAG for over-funded outliers
            if ratio >= 100.0:
                print("ISS "+str(iss)+' with ratio '+str(ratio)+" in "+str((fund,req))+ ", flagged for possible outlier")
                print("Replaced with ratio = 100.0 (over-funded) to prevent inflated ratio from biasing insights")
                ratio =100.0
                print("")

        elif ((fund < req) and (fund > 0.0) and (req > 0.0)):
            #print("UNDER")
            deficit = fund-req
            ratio = deficit/req

        elif (fund == req):
            #print("MET")
            deficit = 0.0
            ratio = 0.0

        rowdefs.append(deficit)
        ratios.append(ratio)
    #print("DEFS: " + str(rowdefs))
    return ratios

In [12]:
#CHECKPOINT
#print(defmatrix2[1:][1:17][2][1:])
#for x in np.arange(16):
   # print(get_deficit_ratios(defmatrix2[1:][1:17][x][1:]))

In [76]:
ratiorows = {}
for item in uniques:
    ratiocount = 0
    sums=[0.0]*12
    newsums = [0.0]*12
    for row in defmatrix2[1:]:
        if row[0] == item:
            ratios = get_deficit_ratios(item, row[1:])
            for value in range(12):
                sums[value]+=ratios[value]
            ratiocount+=1
    for value in range(len(sums)):
        temp = sums[value]/ratiocount
        newsums[value] = temp
    ratiorows[item] = newsums

#for item in uniques:
    #print(str(item) + ': ' + str(ratiorows[item]))
    #print("")

ISS 600 with ratio 341.63829787234044 in (80520.0, 235.0), flagged for possible outlier
Replaced with ratio = 100.0 (over-funded) to prevent inflated ratio from biasing insights

ISS 600 with ratio 152.07926829268294 in (50210.0, 328.0), flagged for possible outlier
Replaced with ratio = 100.0 (over-funded) to prevent inflated ratio from biasing insights

ISS 108 with ratio 419.20168067226894 in (350028.0, 833.0), flagged for possible outlier
Replaced with ratio = 100.0 (over-funded) to prevent inflated ratio from biasing insights

ISS 700 with ratio 20085.11111111111 in (180775.0, 9.0), flagged for possible outlier
Replaced with ratio = 100.0 (over-funded) to prevent inflated ratio from biasing insights



In [77]:
rowvalues = ['F10', 'R10', 'F11', 'R11', 'F12', 'R12', 'F13', 'R13', 'F14', 'R14',
           'F15', 'R15', 'F16', 'R16', 'F17', 'R17', 'F18', 'R18', 'F19', 'R19', 'F20', 'R20', 'F21', 'R21']
rowyears = ['2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021']

In [86]:
keys=[]
for uni in uniques:
    for k in uniqdef.keys():
        if k == uni:
            keys.append(k)

rowtuples = []
rowtuples.append(keys)
for uni in uniques:
    for k in uniqdef.keys():
        if k == uni:
            loc = list(uniqdef.keys()).index(k)
            v = (list(uniqdef.values()))[loc]
            rowtuples.append(v)

In [87]:
keys=[]
for uni in uniques:
    for k in ratiorows.keys():
        if k == uni:
            keys.append(k)

ratiotuples = []
ratiotuples.append(keys)
for uni in uniques:
    for k in ratiorows.keys():
        if k == uni:
            loc = list(ratiorows.keys()).index(k)
            v = (list(ratiorows.values()))[loc]
            ratiotuples.append(v)

In [80]:
with open('newnumbers2.csv', 'w') as f:  # Just use 'w' mode in 3.x
    w = csv.writer(f, delimiter=',', quoting=csv.QUOTE_ALL)
    for k in uniqdef.keys():
        w.writerow(k)
    for v in uniqdef.values():
        w.writerow(v)

In [64]:
with open('ratios.csv', 'w') as f:  # Just use 'w' mode in 3.x
    w = csv.writer(f, delimiter=',', quoting=csv.QUOTE_ALL)
    for k in ratiorows.keys():
        w.writerow(k)
    for v in ratiorows.values():
        w.writerow(v)